# Problem 1(e)

# Forward and Back word verification

In [1]:
import torch

# Define variables as tensors with requires_grad=True to track gradients
x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(-1.0, requires_grad=True)
z = torch.tensor(2.0, requires_grad=True)

# Define the function f
f = x**3 * y + y**3 * z + z**2 - 4

# Perform forward pass: compute value of f
print(f"Function value f = {f.item()}")

# Perform backward pass: compute gradients
f.backward()

# Print gradients (partial derivatives)
print(f"Partial derivative with respect to x: {x.grad.item()}")
print(f"Partial derivative with respect to y: {y.grad.item()}")
print(f"Partial derivative with respect to z: {z.grad.item()}")


Function value f = -3.0
Partial derivative with respect to x: -3.0
Partial derivative with respect to y: 7.0
Partial derivative with respect to z: 3.0


# Problem 2 A verification 

In [12]:

# Set a random seed to get the same result every time
torch.manual_seed(0)

# Define the size of the vectors and matrix
n = 5

# Step 1: Create random matrix J and vectors h, x
J = torch.randn(n, n)  # n x n random matrix
h = torch.randn(n)     # n-element random vector
x = torch.randn(n, requires_grad=True)  # input vector x with gradient tracking enabled

# Step 2: Calculate L using the formula for quadratic and linear terms
L = x @ J @ x + h @ x  # scalar value: x^T J x + h^T x

# Manually calculate gradient of L with respect to x using matrix calculus formula
grad_manual = (J + J.T) @ x + h  # (J + J^T) * x + h

# Step 3: Use PyTorch's automatic differentiation to compute gradient
L.backward()  
grad_auto = x.grad  # get gradient computed by PyTorch

# Step 4: Compute the difference between manual and automatic gradients
difference = torch.norm(grad_manual - grad_auto).item()

# Print results to see if manual calculation matches PyTorch's gradient
print("Manual Gradient:", grad_manual)
print("Automatic Gradient:", grad_auto)
print(f"Difference (norm): {difference:.6f}")

# Check if the difference is small enough
if difference < 1e-4:
    print("Great! The gradients match, so the manual formula is correct.")
else:
    print("Oops! The gradients don't match. Please check the calculation.")



Manual Gradient: tensor([ 2.7621,  2.5385,  6.0925, 11.1068, -4.2827], grad_fn=<AddBackward0>)
Automatic Gradient: tensor([ 2.7621,  2.5385,  6.0925, 11.1068, -4.2827])
Difference (norm): 0.000001
Great! The gradients match, so the manual formula is correct.


#  Problem 2 B 

In [68]:
import torch

# Given problem parameters with h pushing x2, x3 towards -1
J = torch.tensor([[2.0, 0.0, 0.0],
                  [0.0, 2.0, 0.0],
                  [0.0, 0.0, 2.0]])
h = torch.tensor([[0.0], [4.0], [4.0]])
x0 = torch.zeros((3, 1))  # Starting at zeros

alpha = 1e-3 
num_epochs = 10000 

print("J size:", J.size())
print("h size:", h.size())
print("x0 size:", x0.size())

x = x0.clone()
best_value = float('inf')
best_x = None

for epoch in range(num_epochs):
    x_var = torch.nn.Parameter(x, requires_grad=True)
    L = (x_var.t() @ J @ x_var) + (h.t() @ x_var)
    L.backward()
    
    with torch.no_grad():
        x -= alpha * x_var.grad
        x.clamp_(-1, 1)

    if L.item() < best_value:
        best_value = L.item()
        best_x = x.clone()

    if (epoch + 1) % 1000 == 0:
        print(f"Iteration {epoch+1}: L = {L.item():.4f}, x = {x.view(-1).tolist()}")

    x_var.grad.zero_()

print('------------------------------------')
print('Solution:', x)
print(f"\nBest value of L found: {best_value:.4f}")
print(f"Corresponding x: {best_x.view(-1).tolist()}")


J size: torch.Size([3, 3])
h size: torch.Size([3, 1])
x0 size: torch.Size([3, 1])
Iteration 1000: L = -3.9987, x = [0.0, -0.9818307161331177, -0.9818307161331177]
Iteration 2000: L = -4.0000, x = [0.0, -0.9996699690818787, -0.9996699690818787]
Iteration 3000: L = -4.0000, x = [0.0, -0.9999925494194031, -0.9999925494194031]
Iteration 4000: L = -4.0000, x = [0.0, -0.9999925494194031, -0.9999925494194031]
Iteration 5000: L = -4.0000, x = [0.0, -0.9999925494194031, -0.9999925494194031]
Iteration 6000: L = -4.0000, x = [0.0, -0.9999925494194031, -0.9999925494194031]
Iteration 7000: L = -4.0000, x = [0.0, -0.9999925494194031, -0.9999925494194031]
Iteration 8000: L = -4.0000, x = [0.0, -0.9999925494194031, -0.9999925494194031]
Iteration 9000: L = -4.0000, x = [0.0, -0.9999925494194031, -0.9999925494194031]
Iteration 10000: L = -4.0000, x = [0.0, -0.9999925494194031, -0.9999925494194031]
------------------------------------
Solution: tensor([[ 0.0000],
        [-1.0000],
        [-1.0000]])

B